# Transformer by PyTorch (Attention Is All You Need)

![Transformer](fig/transformer.png)

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Preparing Data

In [2]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    """
    Tokenize German text. 
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenize English text.
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [3]:
from torchtext.data import Field, BucketIterator

# Set `batch_first=False` in the `Field`.
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True, batch_first=False)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True, batch_first=False)

In [4]:
from torchtext.datasets import Multi30k

train_data, valid_data, test_data = Multi30k.splits(exts=['.de', '.en'], 
                                                    # fields=[SRC, TRG], 
                                                    fields=[('src', SRC), ('trg', TRG)], 
                                                    root='data/')

In [5]:
print(train_data[0].src)
print(train_data[0].trg)

['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']


In [6]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

len(SRC.vocab), len(TRG.vocab)

(7855, 5893)

In [7]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [8]:
for batch in train_iterator:
    batch_src, batch_src_lens = batch.src
    batch_trg, batch_trg_lens = batch.trg
    break
print(batch_src)
print(batch_src_lens)
print(batch_trg)
print(batch_trg_lens)

tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  5,   5,  43,  ...,   5,  18,  18],
        [ 13,  13, 253,  ...,  13,  30,   0],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]])
tensor([14, 17, 12, 11, 17, 21, 12, 16, 14, 11, 23, 23,  8, 11,  9, 14, 19, 20,
        12, 16,  9, 11, 13, 20, 21, 29, 13, 22, 14, 16, 10,  9, 15, 12, 17, 10,
        14, 22, 17, 20, 23, 23, 12, 17, 15, 19, 17, 15, 16,  7, 14, 15, 16, 12,
        17, 14, 18, 18, 14, 14, 17, 21, 12, 12,  9, 19, 12, 14, 12, 11, 10, 13,
        18, 14,  9, 11, 10, 12, 10, 25, 14, 18, 15, 16, 15, 18, 13,  9, 21, 11,
        20, 12, 13, 14, 14, 17, 10, 13, 18, 30, 14, 12, 13,  9, 10, 15, 13, 10,
        12, 15, 13, 18, 17, 13, 11, 12, 10, 16, 12, 13, 24, 14, 19, 19, 10, 20,
        12, 11])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,   48,  ...,    4,   16,   16],
        [   9,    9,   25,  .

## Building the Model
### Multi-Head Attention

`nn.MultiheadAttention.forward`
* `key_padding_mask` is an binary mask - when the value is `True`, the corresponding value on the (`trg_step` * `src_step`) energy matrix will be filled with `-inf` before passing to `softmax`.  
* `attn_mask` is an additive mask (i.e. the values will be added to the energy matrix before `softmax`). Hence, the value being `-inf` means "masked", and the value being `0` means "not-masked". 
    * This mask aims to prevent attention to certain positions.  
    * A 2D mask will be broadcasted for all the batches while a 3D mask allows to specify a different mask for the entries of each batch. 

In [9]:
SRC_IN_DIM = len(SRC.vocab)
TRG_IN_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
ENC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
DEC_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]


attention = nn.MultiheadAttention(embed_dim=HID_DIM, num_heads=ENC_HEADS, 
                                  dropout=ENC_DROPOUT).to(device)
src_emb = nn.Embedding(SRC_IN_DIM, HID_DIM, padding_idx=ENC_PAD_IDX).to(device)
trg_emb = nn.Embedding(TRG_IN_DIM, HID_DIM, padding_idx=DEC_PAD_IDX).to(device)
# The dropout may cause the sum of attention not equaling 1. 
attention.eval()
src_emb.eval()
trg_emb.eval()

# mask: (batch, src_step)
mask = (batch_src == src_emb.padding_idx).T
# K: (src_step, batch, hid_dim)
K = src_emb(batch_src)
# Q: (trg_step, batch, hid_dim)
Q = trg_emb(batch_trg)

# attened_values: (trg_step, batch, hid_dim)
# attens: (batch, trg_step, src_step)
# attens is the average attention weights over heads
attened_values, attens = attention(Q, K, K, key_padding_mask=mask)

print(batch_src.size())
print(batch_trg.size())
print(attens.size())
print(attened_values.size())

torch.Size([30, 128])
torch.Size([29, 128])
torch.Size([128, 29, 30])
torch.Size([29, 128, 256])


In [10]:
print((attens.sum(dim=-1) - 1).abs().max())
print(((attens == 0) == mask.unsqueeze(1)).all())

tensor(2.3842e-07, grad_fn=<MaxBackward1>)
tensor(True)


### Encoder

In [11]:
encoder_layer = nn.TransformerEncoderLayer(d_model=HID_DIM, nhead=ENC_HEADS, dim_feedforward=ENC_PF_DIM, dropout=ENC_DROPOUT).to(device)

# mask: (batch, src_step)
mask = (batch_src == src_emb.padding_idx).T
# outs: (src_step, batch, hid_dim)
outs = encoder_layer(K, src_key_padding_mask=mask)

print(batch_src.size())
print(outs.size())

torch.Size([30, 128])
torch.Size([30, 128, 256])


In [12]:
encoder = nn.TransformerEncoder(encoder_layer, num_layers=ENC_LAYERS).to(device)

enc_outs = encoder(K, src_key_padding_mask=mask)

print(batch_src.size())
print(enc_outs.size())

torch.Size([30, 128])
torch.Size([30, 128, 256])


### Decoder

In [13]:
# Use `torch.triu` to create the masking matrix
# This is an additive masking matrix
(torch.ones(5, 5) * -np.inf).triu(1)

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

In [14]:
decoder_layer = nn.TransformerDecoderLayer(d_model=HID_DIM, nhead=DEC_HEADS, dim_feedforward=DEC_PF_DIM, dropout=DEC_DROPOUT).to(device)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=DEC_LAYERS).to(device)

# src_mask: (batch, src_step)
src_mask = (batch_src == src_emb.padding_idx).T
# trg_mask: (trg_step, trg_step)
trg_mask = (torch.ones(batch_trg.size(0), batch_trg.size(0), device=device) * -np.inf).triu(1)

dec_outs = decoder(Q, enc_outs, tgt_mask=trg_mask, memory_key_padding_mask=src_mask)

print(batch_src.size())
print(batch_trg.size())
print(dec_outs.size())

torch.Size([30, 128])
torch.Size([29, 128])
torch.Size([29, 128, 256])


### Transformer

In [15]:
transformer = nn.Transformer(d_model=HID_DIM, custom_encoder=encoder, custom_decoder=decoder).to(device)

trans_outs = transformer(K, Q, src_key_padding_mask=src_mask, memory_key_padding_mask=src_mask, tgt_mask=trg_mask)

print(batch_src.size())
print(batch_trg.size())
print(trans_outs.size())

torch.Size([30, 128])
torch.Size([29, 128])
torch.Size([29, 128, 256])


In [16]:
transformer.eval()
print(encoder.training, decoder.training)

enc_outs = encoder(K, src_key_padding_mask=src_mask)
dec_outs = decoder(Q, enc_outs, tgt_mask=trg_mask, memory_key_padding_mask=src_mask)

trans_outs = transformer(K, Q, src_key_padding_mask=src_mask, memory_key_padding_mask=src_mask, tgt_mask=trg_mask)

print((dec_outs == trans_outs).all())

False False
tensor(True)


### Seq2Seq

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, transformer: nn.Transformer, src_voc_dim: int, trg_voc_dim: int, 
                 src_pad_idx: int, trg_pad_idx: int, dropout: float, max_len: int=100):
        super().__init__()
        self.transformer = transformer
        hid_dim = transformer.d_model

        self.src_tok_emb = nn.Embedding(src_voc_dim, hid_dim, padding_idx=src_pad_idx)
        self.src_pos_emb = nn.Embedding(max_len, hid_dim)
        self.trg_tok_emb = nn.Embedding(trg_voc_dim, hid_dim, padding_idx=trg_pad_idx)
        self.trg_pos_emb = nn.Embedding(max_len, hid_dim)

        self.fc = nn.Linear(hid_dim, trg_voc_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = hid_dim ** 0.5

    def forward(self, src: torch.Tensor, trg: torch.Tensor):
        # src: (src_step, batch)
        # trg: (trg_step-1, batch)
        # For the target sequence, the `<eos>` token should be sliced off before passing to the decoder. 
        # As there are no more tokens to be predicted after `<eos>`. 
        trg = trg[:-1]

        # src_mask: (batch, src_step)
        src_mask = (src == self.src_tok_emb.padding_idx).T
        # trg_mask: (trg_step-1, trg_step-1)
        trg_mask = (torch.ones(trg.size(0), trg.size(0), device=trg.device) * -np.inf).triu(1)
        
        # src_embedded: (src_step, batch, hid_dim)
        src_pos = torch.arange(src.size(0), device=src.device).unsqueeze(-1).repeat(1, src.size(1))
        src_embedded = self.dropout(self.src_tok_emb(src)*self.scale + self.src_pos_emb(src_pos))
        # trg_embedded: (trg_step-1, batch, hid_dim)
        trg_pos = torch.arange(trg.size(0), device=trg.device).unsqueeze(-1).repeat(1, trg.size(1))
        trg_embedded = self.dropout(self.trg_tok_emb(trg)*self.scale + self.trg_pos_emb(trg_pos))

        # trans_outs: (trg_step-1, batch, hid_dim)
        trans_outs = transformer(src_embedded, trg_embedded, src_key_padding_mask=src_mask, memory_key_padding_mask=src_mask, tgt_mask=trg_mask)
        # preds: (trg_step-1, batch, trg_voc_dim)
        return self.fc(trans_outs)

In [18]:
model = Seq2Seq(transformer, SRC_IN_DIM, TRG_IN_DIM, ENC_PAD_IDX, DEC_PAD_IDX, ENC_DROPOUT).to(device)
preds = model(batch_src, batch_trg)

print(batch_src.size())
print(batch_trg.size())
print(preds.size())

torch.Size([30, 128])
torch.Size([29, 128])
torch.Size([28, 128, 5893])


In [19]:
# Check if data are mixed across different samples in a batch.
model.eval()
preds_012 = model(batch_src[:, 0:3], batch_trg[:, 0:3])
preds_123 = model(batch_src[:, 1:4], batch_trg[:, 1:4])
(preds_012[:, 1:] == preds_123[:, :2]).all()

tensor(True)

## Training the Model

In [20]:
# This is important...
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name and param.dim() > 1:
            nn.init.xavier_uniform_(param.data)

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# encoder_layer = nn.TransformerEncoderLayer(d_model=HID_DIM, nhead=ENC_HEADS, dim_feedforward=ENC_PF_DIM, dropout=ENC_DROPOUT).to(device)
# encoder = nn.TransformerEncoder(encoder_layer, num_layers=ENC_LAYERS).to(device)
# decoder_layer = nn.TransformerDecoderLayer(d_model=HID_DIM, nhead=DEC_HEADS, dim_feedforward=DEC_PF_DIM, dropout=DEC_DROPOUT).to(device)
# decoder = nn.TransformerDecoder(decoder_layer, num_layers=DEC_LAYERS).to(device)
# transformer = nn.Transformer(d_model=HID_DIM, custom_encoder=encoder, custom_decoder=decoder).to(device)

transformer = nn.Transformer(d_model=HID_DIM, nhead=ENC_HEADS, num_encoder_layers=ENC_LAYERS, num_decoder_layers=DEC_LAYERS, dim_feedforward=ENC_PF_DIM, dropout=ENC_DROPOUT).to(device)
model = Seq2Seq(transformer, SRC_IN_DIM, TRG_IN_DIM, ENC_PAD_IDX, DEC_PAD_IDX, ENC_DROPOUT).to(device)

model.apply(init_weights)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,039,877 trainable parameters


In [21]:
# Initialize Embeddings 
ENC_UNK_IDX = SRC.vocab.stoi[SRC.unk_token]
DEC_UNK_IDX = TRG.vocab.stoi[TRG.unk_token]

model.src_tok_emb.weight.data[ENC_UNK_IDX].zero_()
model.src_tok_emb.weight.data[ENC_PAD_IDX].zero_()
model.trg_tok_emb.weight.data[DEC_UNK_IDX].zero_()
model.trg_tok_emb.weight.data[DEC_PAD_IDX].zero_()

print(model.src_tok_emb.weight[:5, :8])
print(model.trg_tok_emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0191,  0.0185, -0.0017,  0.0217,  0.0171, -0.0182, -0.0117,  0.0023],
        [-0.0071, -0.0174,  0.0138,  0.0183,  0.0117, -0.0171,  0.0182,  0.0244],
        [-0.0045, -0.0020,  0.0159, -0.0162,  0.0187, -0.0206,  0.0117, -0.0254]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.2469e-02, -5.9176e-03, -2.8848e-02, -5.1947e-03, -7.4722e-03,
         -2.3456e-02,  2.8302e-02,  1.5407e-02],
        [-1.8386e-02, -5.4012e-03, -9.2839e-03,  8.3687e-05, -2.2478e-03,
          3.1187e-02,  7.9230e-03,  1.1304e-02],
        [-2.7775e-02,  1.5458e-02, -1.1077e-03,  2.8353e-02,  2.4

In [22]:
loss_func = nn.CrossEntropyLoss(ignore_index=DEC_PAD_IDX, reduction='mean')
# The `lr` is important...
optimizer = optim.AdamW(model.parameters(), lr=0.0005)

In [23]:
def train_epoch(model, iterator, optimizer, loss_func, clip):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        # Forward pass
        batch_src, batch_src_lens = batch.src
        batch_trg, batch_trg_lens = batch.trg
        # preds: (batch, trg_step-1, trg_out_dim)
        preds = model(batch_src, batch_trg)
        
        # Calculate loss
        preds_flattened = preds.view(-1, preds.size(-1))
        batch_trg_flattened = batch_trg[:, 1:].flatten()
        loss = loss_func(preds_flattened, batch_trg_flattened)

        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update weights
        optimizer.step()
        # Accumulate loss
        epoch_loss += loss.item()
    return epoch_loss/len(iterator)

def eval_epoch(model, iterator, loss_func):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            batch_src, batch_src_lens = batch.src
            batch_trg, batch_trg_lens = batch.trg
            # preds: (batch, trg_step-1, trg_out_dim)
            preds = model(batch_src, batch_trg)
            
            # Calculate loss
            preds_flattened = preds.view(-1, preds.size(-1))
            batch_trg_flattened = batch_trg[:, 1:].flatten()
            loss = loss_func(preds_flattened, batch_trg_flattened)
            
            # Accumulate loss and acc
            epoch_loss += loss.item()
    return epoch_loss/len(iterator)

In [ ]:
import time
N_EPOCHS = 10
CLIP = 1
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, loss_func, CLIP)
    valid_loss = eval_epoch(model, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/tut7-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')

In [ ]:
model.load_state_dict(torch.load('models/tut7-model.pt', map_location=device))

valid_loss = eval_epoch(model, valid_iterator, loss_func)
test_loss = eval_epoch(model, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')
print(f'Test Loss: {test_loss:.3f} |  Test PPL: {np.exp(test_loss):7.3f}')

## Check Embeddings
* The Embeddings of `<unk>` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [46]:
print(model.src_tok_emb.weight[:5, :8])
print(model.trg_tok_emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0264, -0.0120,  0.0091, -0.0051,  0.0115,  0.0011,  0.0186,  0.0209],
        [ 0.0058, -0.0132, -0.0267,  0.0209, -0.0261,  0.0227, -0.0162,  0.0059],
        [-0.0115, -0.0093, -0.0107,  0.0117, -0.0101,  0.0137,  0.0233, -0.0235]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0095, -0.0072, -0.0298,  0.0297,  0.0211,  0.0182, -0.0085, -0.0104],
        [ 0.0049, -0.0076,  0.0292, -0.0285,  0.0056, -0.0247,  0.0311,  0.0225],
        [ 0.0157, -0.0015, -0.0078,  0.0214, -0.0129,  0.0232, -0.0124,  0.0095]],
       grad_fn=<SliceBackward>)


## Inference